In [8]:
# Twelvelabs

In [15]:
from typing import List
from dotenv import dotenv_values
from twelvelabs import TwelveLabs
from twelvelabs.types import VideoSegment
from twelvelabs.embed import TasksStatusResponse

config = dotenv_values(".env")

# 1. Initialize the client
client = TwelveLabs(api_key=config["TWELVELABS_API_KEY"])

# 2. Upload a video
task = client.embed.tasks.create(
    model_name="marengo3.0",
    video_url="/Users/pramaka/Desktop/Projects/LangChain/pushup.mp4", # Or use video_file to upload a file from the local file system
    # video_clip_length=5,
    # video_start_offset_sec=30,
    # video_end_offset_sec=60,
    # video_embedding_scope=["clip", "video"]
)
print(f"Created video embedding task: id={task.id}")

# 3. Monitor the status
def on_task_update(task: TasksStatusResponse):
    print(f"  Status={task.status}")

status = client.embed.tasks.wait_for_done(sleep_interval=5, task_id=task.id, callback=on_task_update)
print(f"Embedding done: {status.status}")

# 4. Retrieve the embeddings
task = client.embed.tasks.retrieve(
    task_id=task.id,
    embedding_option=["visual", "audio", "transcription"]
)

# 5. Process the results
def print_segments(segments: List[VideoSegment], max_elements: int = 5):
    for segment in segments:
        print(f"  embedding_scope={segment.embedding_scope} embedding_option={segment.embedding_option} start_offset_sec={segment.start_offset_sec} end_offset_sec={segment.end_offset_sec}")
        first_few = segment.float_[:max_elements]
        print(
            f"  embeddings: [{', '.join(str(x) for x in first_few)}...] (total: {len(segment.float_)} values)"
        )


if task.video_embedding is not None and task.video_embedding.segments is not None:
    print_segments(task.video_embedding.segments)



BadRequestError: headers: {'date': 'Wed, 19 Nov 2025 23:28:13 GMT', 'content-type': 'application/json; charset=UTF-8', 'content-length': '123', 'connection': 'keep-alive', 'content-encoding': 'gzip', 'tl-report': 'backend', 'vary': 'Accept-Encoding', 'x-ratelimit-limit': '50', 'x-ratelimit-remaining': '50', 'x-ratelimit-reset': '1763679051', 'x-ratelimit-used': '0', 'x-trace-id': '7632340184606122143'}, status_code: 400, body: {'code': 'video_file_broken', 'message': 'Unable to process video file. Please check if the file is valid and try again.'}